# Лабораторная работа #3
## Метод стохастического градиентного спуска (SGD) и его модификации
***
_Авторы: Заречнев Алексей, Петрасюк Алексей, Халили Алина, Галимова Ярослава (все - счастливые обитатели группы M3236). Команда "Зелёные шапки"_

[Ссылка на репозиторий](https://github.com/3ELEHblE-LIJanki/Optimization-Methods-lab3#)
***

В этой лабораторной работе требуется реализовать метод __стохастического градиентного спуска__ (или, как мы будем кратко его называть, SGD). От обычного градиентного спуска он отличается тем, что мы считаем не весь градиент, а производную по некоторым направлениям. Иногда - только по одной координате. Иногда - по нескольким сразу. Количество выбранных координат назвается батчем. В ходе этой лабораторной мы будем смотреть на скорость работы метода при разных батчах. 

Благодаря предыдущим лабораторным работам и нашим усилиям у нас уже есть универсальная библиотека `Optimization-lib`, лежащая в основе наших предыдущих работ. В ней можно найти не только реализованные методы нулевого, первого и второго порядка, но также и различные стратегии выбора шага `lrs.py`, которые мы также будем брать во внимание в этой работе.

__Отличительной чертой__ SGD является то, что этот алгоритм предназначен для работы с датасетами с большим количеством параметров. Это значит, что мы изучаем функцию с большим количеством измерений. И если раньше, например, в алгоритме банды четырёх BFGS-е мы использовали оптимизации, котрые укоряют нам подсчёт затратных по времени матриц, то сейчас всё гораздо проще, ведь на каждом шаге для функции с размерностью N мы делаем не N подсчётов, а 1 (или, в случае с Mini-batch GD, некоторое $k \le N$). Да, таким образом мы будем делать больше шагов. Но почему SGD в теории должен быть эффективнее? Потому что раньше изменения функции происходило раз в N вычислений, где N - размерность. Теперь же изменения происходят после каждого вычисления (или, в случае с Mini-batch GD, после k вычислений). В SGD каждое обновление, даже если оно менее точное, сразу корректирует направление поиска, что ускоряет движение к оптимуму, даже если каждый отдельный шаг приближает не так быстро. То есть функция обновляется чаще, а значит, мы решаем сразу несколько проблем:
   1.  Эффективность в высокоразмерных пространствах: не требует вычисления и хранения больших матриц (например, гессиана в случае с методом Ньютона). То есть метод становится ещё и более эффективным по памяти и требуемым ресурсам.
   2.  Уход от локальных минимумов: стохастичность помогает "выпрыгивать" из плоских областей и локальных оптимумов. Да, это не всегда хорошо, если локальным оптимум является ещё и глобальным. Но для мультимодальных фунций это однозначный плюс. А также мы не будем застревать в седловых точках, что тоже хорошо.

Давайте теперь проверим, так ли это на практике. Для анализа данных мы взяли датасет из предложенного сайта. Вот <u>[ссылка](https://archive.ics.uci.edu/dataset/186/wine+quality)</u> на него. А также добавили с библиотеку новый класс, реализующий стохастический градиентный спуск:

Что за функция, которую мы хотим минимизировать? Это функция потерь `Loss function`, которая так или иначе выражает, насколько текущее значение далеко от ожидаемого. Именно поэтому мы хотим её минимизировать. А минимизировать мы будем с помощью подбора коэффициентов. Если говорить простыми словами, то эти коэффициенты показывают, насколько соответствующий параметр важен для определения результата. Например, наш датасет связан с вином и отношением людей к нему. И мы, не прибегая к написанию кода, можем исходя из жизненного опыта и нашей логики предположить, что пол и возраст человека играют большую роль во вкусовых предпочтения, чем, например, наличие домашнего питомца. Но давайте обо всём по порядку. Мы предполагаем, что мы линейно зависим от параметров. То есть наша модель называется __многомерная линейная регрессия__, которую формально можно представить следующим образом:

$
\omega_0 + \omega_1 x^{(1)} + \omega_2 x^{(2)} + \dots + \omega_n x^{(n)} = 
\begin{pmatrix}
\omega_0 & \omega_1 & \dots & \omega_n
\end{pmatrix}
\begin{pmatrix}
1 \\
x^{(1)} \\
\vdots \\
x^{(n)}
\end{pmatrix}
$

Функция потерь для модели:

$
L = \frac{1}{N} \sum_{i=1}^N \left(y_i - \boldsymbol{\omega}^T \mathbf{\hat{x}}_i\right)^2, \quad \text{где } \mathbf{\hat{x}}_i = 
\begin{pmatrix}
1 \\
x_i^{(1)} \\
\vdots \\
x_i^{(n)}
\end{pmatrix}
$

То есть нас интересует квадратичное отклонение от результата. Почему именно квадратичное? Во-первых, само словосочетание "квадратичное отклонение" отсылает нас к теории вероятности и понятии дисперсии. У нас есть много факторов, которые как-то (предполагаем, что независимо) влияют на результат, а значит, согласно центрольное предельной теореме, имеют в среднем нормальное распределение. А ещё - рассматривать квадраты это всегда удобно - нас волнует величина отклонения, то есть модуль, и мы при этом не хотим терять дифференцируемость. Такая функция потерь нас абсолютно устраивает, поэтому работать мы будем с ней. 